# Vector Bundles and Characteristic Classes

This Jupyter notebook illustrate SageMath functionalities regarding vector bundles, mixed differential forms and characteristic classes. The involved tools have been developed through the [SageManifolds](https://sagemanifolds.obspm.fr) project. 

*Author:* **Michael Jung**

A version of SageMath at least equal to 9.0 is required to run this notebook:

In [1]:
version()

'SageMath version 9.1.beta5, Release Date: 2020-02-21'

First we set up the notebook to display math formulas using LaTeX formatting:

In [2]:
%display latex

## Vector Bundles

### Example: The Möbius Bundle

In this section we want to introduce a non-trivial line bundle and explain how it can be applied within `Sage` making use of our implementation.

In [3]:
M = Manifold(1, 'RP1',
             latex_name=r'\mathbb{RP}^1',
             start_index=1,
             structure='topological')
U = M.open_subset('U'); hu.<u> = U.chart()
V = M.open_subset('V'); hv.<v> = V.chart()
M.declare_union(U, V)

The change of coordinates is given by:

In [4]:
u_to_v = hu.transition_map(hv, 1/u,
                           intersection_name='W',
                           restrictions1= u!=0,
                           restrictions2= v!=0)
u_to_v.display()

v = 1/u

In [5]:
W = U.intersection(V)
v_to_u = u_to_v.inverse()
v_to_u.display()

u = 1/v

We define the Möbius bundle in `Sage`:

In [6]:
E = M.vector_bundle(1, 'E'); print(E)

Topological real vector bundle E -> RP1 of rank 1 over the base space 1-dimensional topological manifold RP1


Let us state the two trivializations:

In [7]:
psiU = E.trivialization('psiU', latex_name=r'\psi_U', domain=U)
psiU

Trivialization (psiU, E|_U)

In [8]:
psiV = E.trivialization('psiV', latex_name=r'\psi_V', domain=V)
psiV

Trivialization (psiV, E|_V)

Next we declare the transition map between $\psi_U$ and $\psi_V$:

In [9]:
transf = psiU.transition_map(psiV, [[u]]); transf

Transition map from Trivialization (psiU, E|_U) to Trivialization (psiV, E|_V)

Of course each trivialization induces a local frame which we can get by the following command:

In [10]:
eU = psiU.frame(); eU

Trivialization frame (E|_U, ((psiU^*e_1)))

The notation $\left(\psi_U^* e_{ 1 }\right)$ stands for the local section on $U$ given by $p \mapsto \psi^{-1}_U(p,e_1)$, where $e_1$ is the standard basis of $\mathbb{R}$. Similarly, we have for $\psi_V$:

In [11]:
eV = psiV.frame(); eV

Trivialization frame (E|_V, ((psiV^*e_1)))

The corresponding bundle automorphism $\psi_U^{-1} \circ \psi_V:\left. E \right|_{W} \to \left. E \right|_{W}$ translating $\left(\psi_V^* e_{ 1 }\right)$ into $\left(\psi_U^* e_{ 1 }\right)$ can be easily returned:

In [12]:
transf.automorphism()

Automorphism psiU^(-1)*psiV of the Free module C^0(W;E) of sections on the Open subset W of the 1-dimensional topological manifold RP1 with values in the real vector bundle E of rank 1

This is an instance of `FreeModuleAutomorphism`:

In [13]:
from sage.tensor.modules.free_module_automorphism import FreeModuleAutomorphism
isinstance(transf.automorphism(), FreeModuleAutomorphism)

True

We can even get its determinant which is a scalar field on the intersection $W = U \cap V$:

In [14]:
transf.det().display()

det(psiU^(-1)*psiV): W --> R
   u |--> u
   v |--> 1/v

We can see that the determinant is negative if $u<0$. It is therefore reasonable to suspect that the vector bundle $E$ is not orientable. This is indeed true. As a consequence, $E$ is not trivial and each global section must vanish somewhere. To exemplify this, we define the corresponding section module over $\mathbb{RP}^1$:

In [15]:
C0 = E.section_module(); C0

Module C^0(RP1;E) of sections on the 1-dimensional topological manifold RP1 with values in the real vector bundle E of rank 1

We can see that `Sage` rejects $C^{0}(\mathbb{RP}^1;E)$ as a free module:

In [16]:
from sage.tensor.modules.finite_rank_free_module import FiniteRankFreeModule
isinstance(C0, FiniteRankFreeModule)

False

This is because there is no global frame that `Sage` knows about:

In [17]:
E.is_manifestly_trivial()

False

On the contrary, the section module over $U$ must be free:

In [18]:
C0U = E.section_module(domain=U); C0U

Free module C^0(U;E) of sections on the Open subset U of the 1-dimensional topological manifold RP1 with values in the real vector bundle E of rank 1

And indeed, it is:

In [19]:
from sage.tensor.modules.finite_rank_free_module import FiniteRankFreeModule
isinstance(C0U, FiniteRankFreeModule)

True

We start with some concrete computations and therefore define a section on $U$:

In [20]:
sU = E.section(name='sigma', latex_name=r'\sigma', domain=U)
sU[eU,1] = (1-u)/(1+u^2)
sU.display()

sigma = -(u - 1)/(u^2 + 1) (psiU^*e_1)

This local section lives in the free module $C^0(U;E)$:

In [21]:
sU in C0U

True

We can perform a change of frame on the subset $W$:

In [22]:
sU.display(eV.restrict(W), hv.restrict(W))

sigma = (v - 1)/(v^2 + 1) (psiV^*e_1)

This expression is obviously well-defined on the whole subset $V$. Hence, we can continue this section onto $\mathbb{RP}^1$:

In [23]:
s = E.section(name='sigma', latex_name=r'\sigma')
s.set_restriction(sU)
s.add_comp_by_continuation(eV, W)
s.display(eV)

sigma = (v - 1)/(v^2 + 1) (psiV^*e_1)

The corresponding continuation is indeed an element of $C^{0}(\mathbb{RP}^1;E)$:

In [24]:
s in C0

True

Let us define another global section in $E$:

In [25]:
t = E.section(name='tau', latex_name=r'\tau')
t[eV,1] = (3-v^2)/(1+v^4)
t.add_comp_by_continuation(eU, W)
t.display(eU)

tau = (3*u^3 - u)/(u^4 + 1) (psiU^*e_1)

Now, $\sigma$ and $\tau$ can be added pointwise:

In [26]:
r = (s + t); r.display(eU)

sigma+tau = (2*u^5 + u^4 + 2*u^3 - 2*u + 1)/(u^6 + u^4 + u^2 + 1) (psiU^*e_1)

In [27]:
r.display(eV)

sigma+tau = (v^5 - 2*v^4 + 2*v^2 + v + 2)/(v^6 + v^4 + v^2 + 1) (psiV^*e_1)

Since $\sigma + \tau$ is again a well-defined continuous section on $E$, it must vanish at some point. We want to check this by solving an equation:

In [28]:
sol = solve(r[eU,1,hu].expr() == 0, u, solution_dict=True)
sol

[{u: -1}]

Let us investigate what happens at this particular point $p\in \mathbb{RP}^1$ determined by $u=-1$:

In [29]:
p = M.point([sol[0][u]], name='p', chart=hu); print(p)

Point p on the 1-dimensional topological manifold RP1


The corresponding section $\sigma$ evaluated at $p$ is an element of the fiber $E_p$:

In [30]:
print(s.at(p))

Vector sigma in the fiber of E at Point p on the 1-dimensional topological manifold RP1


Concretely we have:

In [31]:
s.at(p).display(basis=eU.at(p))

sigma = (psiU^*e_1)

For $\tau$ we similarly obtain:

In [32]:
t.at(p).display(basis=eU.at(p))

tau = -(psiU^*e_1)

As expected, the sum vanishes at $p$:

In [33]:
r.at(p).display(basis=eU.at(p))

sigma+tau = 0

### Example: Tensor Bundles over $\mathbb{S}^2$

We define $U \subset \mathbb{S}^2$ to be the complement of the meridian lying in the upper $x$-$z$-plane for $x\geq 0$. Similarly $V \subset \mathbb{S}^2$ defines the complement of the meridian going through the $x$-$y$-plane for $x \leq 0$. In the language of `Sage` we write:

In [34]:
M = Manifold(2, name='S^2', latex_name=r'\mathbb{S}^2')
U = M.open_subset('U'); V = M.open_subset('V')
M.declare_union(U,V) # M is the union of U and V

The corresponding tangent bundle can be returned by the following command:

In [35]:
TM = M.tangent_bundle(); TM

Tangent bundle TS^2 over the 2-dimensional differentiable manifold S^2

However for now, we are interested in the parallelizable subset $U \subset \mathbb{S}^2$:

In [36]:
TU = U.tangent_bundle(); print(TU)

Tangent bundle TU over the Open subset U of the 2-dimensional differentiable manifold S^2


Trivializations entirely fall back on the class `DiffChart` at this stage. To demonstrate how it is done, we introduce spherical coordinates on the subset $U$:

In [37]:
c_spher.<th,ph> = TU.trivialization(r'th:(0,pi):\theta ph:(0,2*pi):\phi')

To demonstrate the pullback of tensor bundles, we define the Euclidean space $\mathbb{R}^3$ and introduce a differential map $\varphi: \mathbb{S}^2 \to \mathbb{R}^3$ given by the embedding of $\mathbb{S}^2$ into $\mathbb{R}^3$:

In [38]:
R = Manifold(3, 'R^3', r'\mathbb{R}^3')
c_cart.<x,y,z> = R.chart()  # Cartesian coord. on R^3
phi = U.diff_map(R, (sin(th)*cos(ph), sin(th)*sin(ph), cos(th)),
                 name='phi', latex_name=r'\varphi'); print(phi)

Differentiable map phi from the Open subset U of the 2-dimensional differentiable manifold S^2 to the 3-dimensional differentiable manifold R^3


Let us fix the point $p$ in $U \subset \mathbb{S}^2$ determined by $(\frac{\pi}{2},\pi)$ in spherical coordinates:

In [39]:
p = U.point((pi/2, pi), name='p'); print(p)	

Point p on the 2-dimensional differentiable manifold S^2


We can evaluate $\varphi$ at this point $p$: 

In [40]:
phi(p).coord(c_cart)

(-1, 0, 0)

We get the corresponding pullback tensor bundles by stating $\varphi$ as the destination map:

In [41]:
phiT11U = U.tensor_bundle(1,1, dest_map=phi); phiT11U

Tensor bundle phi^*T^(1,1)R^3 over the Open subset U of the 2-dimensional differentiable manifold S^2 along the Differentiable map phi from the Open subset U of the 2-dimensional differentiable manifold S^2 to the 3-dimensional differentiable manifold R^3

More precisely:

In [42]:
print(phiT11U)

Tensor bundle phi^*T^(1,1)R^3 over the Open subset U of the 2-dimensional differentiable manifold S^2 along the Differentiable map phi from the Open subset U of the 2-dimensional differentiable manifold S^2 to the 3-dimensional differentiable manifold R^3


We see that sections completely fall back on the preexisting implementation of tensor fields:

In [43]:
phiT11U.section_module() is U.tensor_field_module((1,1), dest_map=phi)

True

The fiber at $p$ is given by the space of $(1,1)$-tensors of the tangent space over $\mathbb{R}^3$ at $\varphi(p)$:

In [44]:
phiT11U.fiber(p)

Free module of type-(1,1) tensors on the Tangent space at Point phi(p) on the 3-dimensional differentiable manifold R^3

Since $\mathbb{R}^3$ is parallelizable, the pullback tensor bundle $\varphi^* T^{(1,1)}\mathbb{R}^3\to U$ must be trivial:

In [45]:
phiT11U.is_manifestly_trivial()

True

Hence it comes with a frame naturally induced by the pullback:

In [46]:
phiT11U.frames()

[Vector frame (U, (d/dx,d/dy,d/dz)) with values on the 3-dimensional differentiable manifold R^3]

Strictly speaking, this is a frame of $\varphi^* T\mathbb{R}^3$ rather than $\varphi^* T^{(1,1)}\mathbb{R}^3$. But remember that all frames in the tensor bundle can be retrieved from frames in the tangent bundle. Thus, there is no loss of generality here. We can extract our frame by applying index operations on the returned list:

In [47]:
print(phiT11U.frames()[0])

Vector frame (U, (d/dx,d/dy,d/dz)) with values on the 3-dimensional differentiable manifold R^3


## Mixed Differential Forms

### Examples

We briefly demonstrate the capabilities of our implementation, and start by declaring the manifold $M=\mathbb{R}^2$:

In [48]:
M = Manifold(2, 'R^2', latex_name=r'\mathbb{R}^2')
X.<x,y> = M.chart()

We define the corresponding spaces of differential forms:

In [49]:
Omega0 = M.diff_form_module(0); print(Omega0)
Omega1 = M.diff_form_module(1); print(Omega1)
Omega2 = M.diff_form_module(2); print(Omega2)

Algebra of differentiable scalar fields on the 2-dimensional differentiable manifold R^2
Free module Omega^1(R^2) of 1-forms on the 2-dimensional differentiable manifold R^2
Free module Omega^2(R^2) of 2-forms on the 2-dimensional differentiable manifold R^2


The algebra of mixed forms is returned by a simple command:

In [50]:
Omega = M.mixed_form_algebra(); print(Omega)

Graded algebra Omega^*(R^2) of mixed differential forms on the 2-dimensional differentiable manifold R^2


It belongs to the category of graded algebras over the symbolic ring:

In [51]:
print(Omega.category())

Category of graded algebras over Symbolic Ring


Before proceeding with mixed forms, let us first declare some differential forms:

In [52]:
f = M.scalar_field(name='f')
omega1 = M.diff_form(1, name='omega_1', latex_name=r'\omega_1')
omega2 = M.diff_form(2, name='omega_2', latex_name=r'\omega_2')
eta = M.diff_form(1, name='eta', latex_name=r'\eta')

In the next step, we provide some expressions in local coordinates:

In [53]:
f.set_expr(x^2)
omega1[:] = y, 2*x
omega2[0,1] = 4*x^3
eta[:] = x, y

In [54]:
f.display()

f: R^2 --> R
   (x, y) |--> x^2

In [55]:
omega1.display()

omega_1 = y dx + 2*x dy

In [56]:
omega2.display()

omega_2 = 4*x^3 dx/\dy

In [57]:
eta.display()

eta = x dx + y dy

The category framework of `Sage` captures the entire setup:

In [58]:
all([f in Omega0,
     f in Omega,
     omega1 in Omega1,
     omega1 in Omega,
     omega2 in Omega2,
     omega2 in Omega])

True

Now, let us define a mixed form:

In [59]:
A = M.mixed_form(name='A'); print(A)

Mixed differential form A on the 2-dimensional differentiable manifold R^2


It shall consist of the differential forms $f, \omega_1, \omega_2$. The forms are assigned by using index operations:

In [60]:
A[:] = [f,omega1,omega2]; A.display()

A = f + omega_1 + omega_2

In [61]:
A.display_expansion()

A = [x^2] + [y dx + 2*x dy] + [4*x^3 dx/\dy]

As we can see, the output is sorted by degree. Notice that the forms stored in `A` are given by the very same instances we declared beforehand:

In [62]:
all([A[0] is f,
     A[1] is omega1,
     A[2] is omega2])

True

If that behavior is unwanted, a copy can be made which has the very same expressions in local coordinates but has stored entirely new instances:

In [63]:
Aclone = A.copy()
any(Aclone[k] is A[k] for k in Omega.irange())

False

In [64]:
all(Aclone[k] == A[k] for k in Omega.irange())

True

Let us perform some computations and define another mixed form:

In [65]:
B = M.mixed_form([2,eta,0], name='B'); B.display_expansion()

B = [2] + [x dx + y dy] + [0]

The multiplication is executed degree wise:

In [66]:
all((A * B)[k] == sum(A[j].wedge(B[k - j])
                      for j in range(k + 1))
    for k in Omega.irange())

True

In [67]:
(A * B).display_expansion()

A/\B = [2*x^2] + [(x^3 + 2*y) dx + (x^2*y + 4*x) dy] + [(8*x^3 - 2*x^2 + y^2) dx/\dy]

This particular example is also convenient to demonstrate that the multiplication given by the wedge product is in general neither commutative nor anticommutative:

In [68]:
(B * A).display_expansion()

B/\A = [2*x^2] + [(x^3 + 2*y) dx + (x^2*y + 4*x) dy] + [(8*x^3 + 2*x^2 - y^2) dx/\dy]

Finally, let us compute the exterior derivative:

In [69]:
dA = A.exterior_derivative(); dA.display_expansion()

dA = [0] + [2*x dx] + [dx/\dy]

## Characteristic Classes

### Example: Chern Character over Minkowski Space

We want to exemplify the usage of characteristic classes within Sage by computing the Chern character form $\mathrm{ch}(E, \nabla^E)$ on a complex trivial line bundle $E$ over the 2-dimensional Minkowski space $M$ equipped with a bundle connection $\nabla^E$. We start with the general setup:

In [70]:
M = Manifold(2, 'M', structure='Lorentzian')
X.<t,x> = M.chart()
E = M.vector_bundle(1, 'E', field='complex'); print(E)

Differentiable complex vector bundle E -> M of rank 1 over the base space 2-dimensional Lorentzian manifold M


To trivialize the vector bundle $E$, we fix a global frame $e$:

In [71]:
e = E.local_frame('e') # trivialize

Let us declare an $\mathrm{U}(1)$-connection $\nabla^E$ on $E$ given by an electromagnetic potential $A(t)$:

In [72]:
nab = E.bundle_connection('nabla^E', latex_name=r'\nabla^E')
A = function('A')

The corresponding connection form $\omega$ turns out as:

In [73]:
omega = M.one_form(name='omega', latex_name=r'\omega')
omega[1] = I*A(t)
omega.display()

omega = I*A(t) dx

Let us put this into the connection:

In [74]:
nab.set_connection_form(0, 0, omega)

Notice that the Chern character $\mathrm{ch}(E)$ is already predefined in the system. We can get it by the following command:

In [75]:
ch = E.characteristic_class('ChernChar'); print(ch)

Characteristic class ch of additive type associated to e^x on the Differentiable complex vector bundle E -> M of rank 1 over the base space 2-dimensional Lorentzian manifold M


The computation of the corresponding Chern character form $\mathrm{ch}(E,\nabla^E)$ can be invoked by the method \texttt{get\_form}:

In [76]:
ch_form = ch.get_form(nab)
ch_form.display_expansion()

ch(E, nabla^E) = [1] + [0] + [1/2*d(A)/dt/pi dt/\dx]

We can see that the resulting 2-form coincides with the Faraday tensor divided by $2 \pi$ as expected.

### Example: Chern Class of the Tautological Line Bundle

We start our computation by initializing the complex projective space as 2-dimensional real manifold with coordinates on $U:= \mathbb{CP}^1 \setminus \left\{ [1:0] \right\}$:

In [77]:
M = Manifold(2, 'CP^1', start_index=1)
U = M.open_subset('U')
c_cart.<x,y> = U.chart() # [1:x+I*y]

For the sake of convenience, we additionally declare the complex coordinates $z$ and $\bar{z}$ on $U$:

In [78]:
c_comp.<z, zbar> = U.chart(r'z:z zbar:\bar{z}')
cart_to_comp = c_cart.transition_map(c_comp, (x+I*y, x-I*y))
cart_to_comp.display()

z = x + I*y
zbar = x - I*y

In [79]:
comp_to_cart = cart_to_comp.inverse()
comp_to_cart.display()

x = 1/2*z + 1/2*zbar
y = -1/2*I*z + 1/2*I*zbar

Now, we are ready to construct the tautological line bundle $\gamma_1$:

In [80]:
E = M.vector_bundle(1, 'gamma_1',
                    latex_name=r'\gamma_1',
                    field='complex')

Furthermore we declare a local frame $e$ on $U$ naturally given by $[z:1] \mapsto \left(\begin{smallmatrix} z \\ 1 \end{smallmatrix}\right)$:

In [81]:
e = E.local_frame('e', domain=U)

To compute the Chern class, we still need a connection. The tautological line bundle inherits a Hermitian metric from the overlying trivial bundle $\mathbb{C}^2 \times \mathbb{CP}^1$:

In [82]:
nab = E.bundle_connection('nabla', latex_name=r'\nabla')
omega = U.one_form(name='omega')
omega[c_comp.frame(), 1, c_comp] = zbar/(1+z*zbar)
nab.set_connection_form(1, 1, omega, frame=e)

It is time to initialize $c(\gamma_1)$. Fortunately `Sage` already knows the Chern class:

In [83]:
c = E.characteristic_class('Chern'); print(c)

Characteristic class c of multiplicative type associated to x + 1 on the Differentiable complex vector bundle gamma_1 -> CP^1 of rank 1 over the base space 2-dimensional differentiable manifold CP^1


Let the machinery do its work:

In [84]:
c_form = c.get_form(nab)
c_form.display_expansion(c_comp.frame(), chart=c_comp)

c(gamma_1, nabla) = [1] + [0] + [1/2*I/(pi + pi*z^2*zbar^2 + 2*pi*z*zbar) dz/\dzbar]

Since this particular representation is defined outside a set of measure zero, we can compute its integral over $\mathbb{CP}^1$ in real coordinates:

In [85]:
integrate(integrate(c_form[2][[1,2]].expr(c_cart), x, -infinity, infinity).full_simplify(), y, -infinity, infinity)

1

The result shows that $c_1(\gamma_1)$ generates the second integer cohomology $H^2(\mathbb{CP}^1, \mathbb{Z})$.

### Example: Euler Class of $\mathbb{S}^2$

In this example, we want to compute the Euler class of the 2-sphere $\mathbb{S}^2 \subset \mathbb{R}^3$. As usual, we cover $\mathbb{S}^2$ by two parallelizable open subsets $U:=\mathbb{S}^2 \setminus \{(0,0,1)\}$ and $V:=\mathbb{S}^2 \setminus \{(0,0,-1)\}$ for which the point $(0,0,1)$ is identified with the north pole. We state stereographic coordinates in `Sage`: 

In [86]:
M = Manifold(2, name='S^2', latex_name=r'\mathbb{S}^2',
             structure='Riemannian', start_index=1)
U = M.open_subset('U') ; V = M.open_subset('V')
M.declare_union(U,V)   # M is the union of U and V
stereoN.<x,y> = U.chart()
stereoS.<xp,yp> = V.chart("xp:x' yp:y'")
N_to_S = stereoN.transition_map(stereoS,
                                (x/(x^2+y^2), y/(x^2+y^2)),
                                intersection_name='W',
                                restrictions1= x^2+y^2!=0,
                                restrictions2= xp^2+yp^2!=0)
S_to_N = N_to_S.inverse()

Next we define the tangent bundle and its local frames induced by the charts:

In [87]:
eU = stereoN.frame(); eV = stereoS.frame()
TM = M.tangent_bundle()

The Euler class is also one of the predefined classes. Thus, we can easily get it from the tangent bundle's instance:

In [88]:
e_class = TM.characteristic_class('Euler'); print(e_class)

Characteristic class e of Pfaffian type associated to x on the Tangent bundle TS^2 over the 2-dimensional Riemannian manifold S^2


To compute a form representing the Euler class, we need to state a suitable connection first. Here we want to use the Levi-Civita connection induced by the standard metric. This is simply given by the pullback of the Euclidean scalar product of the ambient space $\mathbb{R}^3$ along the canonical embedding $\iota: \mathbb{S}^2 \hookrightarrow \mathbb{R}^3$. Let us define the ambient space $\mathbb{R}^3$ and its Euclidean scalar product $h$:

In [89]:
E = Manifold(3, 'R^3', latex_name=r'\mathbb{R}^3', start_index=1)
cart.<X,Y,Z> = E.chart()
h = E.metric('h')
h[1,1], h[2,2], h[3, 3] = 1, 1, 1
h.display()

h = dX*dX + dY*dY + dZ*dZ

On that account, we declare the embedding $\iota: \mathbb{S}^2 \hookrightarrow \mathbb{R}^3$ in stereographic coordinates when one considers its projection from the north pole $(0, 0, 1)$ to the equatorial plane $Z=0$:

In [90]:
iota = M.diff_map(E, {(stereoN, cart):
                      [2*x/(1+x^2+y^2), 2*y/(1+x^2+y^2),
                       (1-x^2-y^2)/(1+x^2+y^2)],
                      (stereoS, cart):
                      [2*xp/(1+xp^2+yp^2), 2*yp/(1+xp^2+yp^2),
                       (xp^2+yp^2-1)/(1+xp^2+yp^2)]},
                  name='iota', latex_name=r'\iota')
iota.display()

iota: S^2 --> R^3
on U: (x, y) |--> (X, Y, Z) = (2*x/(x^2 + y^2 + 1), 2*y/(x^2 + y^2 + 1), -(x^2 + y^2 - 1)/(x^2 + y^2 + 1))
on V: (xp, yp) |--> (X, Y, Z) = (2*xp/(xp^2 + yp^2 + 1), 2*yp/(xp^2 + yp^2 + 1), (xp^2 + yp^2 - 1)/(xp^2 + yp^2 + 1))

We can define the standard metric $g$ on $\mathbb{S}^2$ by setting it as the pullback metric $\iota^*h$:

In [91]:
g = M.metric()
g.set(iota.pullback(h))
g[1,1].factor(); g[2,2].factor() # simplifications
g.display()

g = 4/(x^2 + y^2 + 1)^2 dx*dx + 4/(x^2 + y^2 + 1)^2 dy*dy

The corresponding Levi-Civita connection is computed automatically:

In [92]:
nab = g.connection()

Since we have found the desired Levi-Civita connection, we want to compute the associated curvature forms and store them in a `Python` list:

In [93]:
cmatrix_U = [[nab.curvature_form(i,j,eU) for j in TM.irange()]
              for i in TM.irange()]
cmatrix_V = [[nab.curvature_form(i,j,eV) for j in TM.irange()]
              for i in TM.irange()]

Fortunately, the curvature form matrices are already skew-symmetric:

In [94]:
for i in range(TM.rank()):
    for j in range(TM.rank()):
        show(cmatrix_U[i][j].display())

curvature (1,1) of connection nabla_g w.r.t. Coordinate frame (U, (d/dx,d/dy)) = 0

curvature (1,2) of connection nabla_g w.r.t. Coordinate frame (U, (d/dx,d/dy)) = 4/(x^4 + y^4 + 2*(x^2 + 1)*y^2 + 2*x^2 + 1) dx/\dy

curvature (2,1) of connection nabla_g w.r.t. Coordinate frame (U, (d/dx,d/dy)) = -4/(x^4 + y^4 + 2*(x^2 + 1)*y^2 + 2*x^2 + 1) dx/\dy

curvature (2,2) of connection nabla_g w.r.t. Coordinate frame (U, (d/dx,d/dy)) = 0

In [95]:
for i in range(TM.rank()):
    for j in range(TM.rank()):
        show(cmatrix_V[i][j].display())

curvature (1,1) of connection nabla_g w.r.t. Coordinate frame (V, (d/dxp,d/dyp)) = 0

curvature (1,2) of connection nabla_g w.r.t. Coordinate frame (V, (d/dxp,d/dyp)) = 4/(xp^4 + yp^4 + 2*(xp^2 + 1)*yp^2 + 2*xp^2 + 1) dxp/\dyp

curvature (2,1) of connection nabla_g w.r.t. Coordinate frame (V, (d/dxp,d/dyp)) = -4/(xp^4 + yp^4 + 2*(xp^2 + 1)*yp^2 + 2*xp^2 + 1) dxp/\dyp

curvature (2,2) of connection nabla_g w.r.t. Coordinate frame (V, (d/dxp,d/dyp)) = 0

Hence we can put them into a dictionary and apply the algorithm:

In [96]:
cmatrices = {eU: cmatrix_U, eV: cmatrix_V}
e_class_form = e_class.get_form(nab, cmatrices)
e_class_form.display_expansion()

e(TS^2, nabla_g) = [0] + [0] + [2/(pi + pi*x^4 + pi*y^4 + 2*pi*x^2 + 2*(pi + pi*x^2)*y^2) dx/\dy]

We want to compute the Euler characteristic of $\mathbb{S}^2$ now. This can be achieved by integrating the top form over $\mathbb{S}^2$. But since $U$ and $\mathbb{S}^2$ differ only by a point and therefore a set of measure zero, it is enough to integrate over the subset $U$:

In [97]:
integrate(integrate(e_class_form[2][[1,2]].expr(), x, -infinity, infinity).simplify_full(), y, -infinity, infinity)

2

We have eventually obtained the Euler characteristic of $\mathbb{S}^2$.

### Example: $\hat{A}$-Class of Lorentzian Foliation of Berger Spheres

We consider the space of unit quaternions $\mathbb{S}^3 \subset \mathbb{R}^4 \cong \mathbb{H}$, where $\mathbb{H}$ is endowed with the canonical basis $(\mathbf{1}, \mathbf{i}, \mathbf{j}, \mathbf{k})$. It turns out that $\mathbb{S}^3$ admits a global frame $(\varepsilon_1, \varepsilon_2, \varepsilon_3)$ so that we observe $\mathbb{S}^3$ to be a parallelizable manifold.

We introduce the following smooth family of so-called *Berger metrics*:
\begin{align*}	
	g_t = a(t)^{2} \; \varepsilon^{1}\otimes \varepsilon^{1} +\varepsilon^{2}\otimes \varepsilon^{2} +\varepsilon^{3}\otimes \varepsilon^{3}.
\end{align*}

This family can be used to define a globally hyperbolic manifold $M=\mathbb{R} \times \mathbb{S}^3$ equipped with the Lorentzian metric $g = - {\mathrm{d} t}^2 + g_t$ and hence foliated by Berger spheres.

In the following we compute the $\hat{A}$-form of the corresponding Levi-Civita connection $\nabla_g$. We start the computation by declaring the Lorentzian manifold first:

In [98]:
M = Manifold(4, 'M', structure='Lorentzian'); print(M)

4-dimensional Lorentzian manifold M


We cover $M$ by two open subsets defined as $U := \mathbb{R} \times \left( \mathbb{S}^3 \setminus \{ -\mathbf{1} \} \right)$ and $V := \mathbb{R} \times \left( \mathbb{S}^3 \setminus \{ \mathbf{1} \} \right)$:

In [99]:
U = M.open_subset('U'); V = M.open_subset('V')
M.declare_union(U,V)

We need to impose coordinates on $M$ and use stereographic projections with respect to the foliated 3-sphere:

In [100]:
stereoN.<t,x,y,z> = U.chart()
stereoS.<tp,xp,yp,zp> = V.chart("tp:t' xp:x' yp:y' zp:z'")
N_to_S = stereoN.transition_map(stereoS,
                                (t, x/(x^2+y^2+z^2),
                                    y/(x^2+y^2+z^2),
                                    z/(x^2+y^2+z^2)),
                                intersection_name='W',
                                restrictions1= x^2+y^2+z^2!=0,
                                restrictions2= xp^2+yp^2+zp^2!=0)
W = U.intersection(V)
S_to_N = N_to_S.inverse()
N_to_S.display()

tp = t
xp = x/(x^2 + y^2 + z^2)
yp = y/(x^2 + y^2 + z^2)
zp = z/(x^2 + y^2 + z^2)

From above, we know that $M$ admits a global frame $(\varepsilon_0, \varepsilon_1, \varepsilon_2, \varepsilon_3)$:

In [101]:
E = M.vector_frame('E', latex_symbol=r'\varepsilon')
E_U = E.restrict(U); E_U

Vector frame (U, (E_0,E_1,E_2,E_3))

The vector field $\varepsilon_0$ is simply given by $\frac{\partial}{\partial t}$. To obtain the global vector frame $(\varepsilon_1, \varepsilon_2, \varepsilon_3)$ on $\mathbb{S}^3$ in stereographic coordinates, a computation within `Sage` is performed in the [this](https://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_sphere_S3_vectors.ipynb) Jupyter-Notebook. This is done by embedding $\mathbb{S}^3$ into $\mathbb{R}^4 \cong \mathbb{H}$, endowing it with the quaternionic structure. This eventually leads to:

In [102]:
E_U[0][:] = [1,0,0,0]
E_U[1][:] = [0, (x^2-y^2-z^2+1)/2, x*y+z, x*z-y]
E_U[2][:] = [0, x*y-z, (1-x^2+y^2-z^2)/2, x+y*z]  
E_U[3][:] = [0, x*z+y, y*z-x, (1-x^2-y^2+z^2)/2]

In [103]:
for i in M.irange():
    show(E_U[i].display())

E_0 = d/dt

E_1 = (1/2*x^2 - 1/2*y^2 - 1/2*z^2 + 1/2) d/dx + (x*y + z) d/dy + (x*z - y) d/dz

E_2 = (x*y - z) d/dx + (-1/2*x^2 + 1/2*y^2 - 1/2*z^2 + 1/2) d/dy + (y*z + x) d/dz

E_3 = (x*z + y) d/dx + (y*z - x) d/dy + (-1/2*x^2 - 1/2*y^2 + 1/2*z^2 + 1/2) d/dz

To ensure evaluations in this particular frame, we must communicate the change-of-frame formula to `Sage`:

In [104]:
P = U.automorphism_field()
for i in M.irange():
    for j in M.irange():
        P[j,i] = E_U[i][j]

In [105]:
U.set_change_of_frame(stereoN.frame(), E_U, P)

The subset $U$ differs from $M$ only by a one dimensional slit, and is therefore dense in $M$. As we know that $\left(\varepsilon_{0},\varepsilon_{1},\varepsilon_{2},\varepsilon_{3}\right)$ defines a \emph{global} frame, its components can be easily and uniquely extended to all of $M$. For this, we use the method `add_comp_by_continuation`:

In [106]:
for i in M.irange():
    E[i].add_comp_by_continuation(stereoS.frame(), W)

And again, we declare the change of frame:

In [107]:
P = V.automorphism_field()
for i in M.irange():
    for j in M.irange():
        P[j,i] = E.restrict(V)[i][j]

In [108]:
V.set_change_of_frame(stereoS.frame(), E.restrict(V), P)

In order to reduce the computation time, we examine the $\hat{A}$-class on the open subset $U \subset M$ first. The final result can be obtained by continuation. For this purpose we define the tangent bundle over $U$:

In [109]:
TU = U.tangent_bundle(); print(TU)

Tangent bundle TU over the Open subset U of the 4-dimensional Lorentzian manifold M


Notice that the $\hat{A}$-class is already predefined:

In [110]:
A = TU.characteristic_class('AHat'); A

Characteristic class A^ of multiplicative type associated to 1/2*sqrt(x)/sinh(1/2*sqrt(x)) on the Tangent bundle TU over the Open subset U of the 4-dimensional Lorentzian manifold M

Its holomorphic function is given by:

In [111]:
A.function()

1/2*sqrt(x)/sinh(1/2*sqrt(x))

We are ready to define the Berger metric, at least on the subset $U$:

In [112]:
a = function('a')

In [113]:
g = U.metric()
g.add_comp(E_U)[0, 0] = - 1
g.add_comp(E_U)[1, 1] = a(t)^2
g.add_comp(E_U)[2, 2] = 1
g.add_comp(E_U)[3, 3] = 1
g.display(E_U)

g = -E^0*E^0 + a(t)^2 E^1*E^1 + E^2*E^2 + E^3*E^3

The corresponding connection is automatically computed by `Sage`:

In [114]:
nab = g.connection(); nab

Levi-Civita connection nabla_g associated with the Lorentzian metric g on the Open subset U of the 4-dimensional Lorentzian manifold M

Finally, we perform the computation of the $\hat{A}$-form with respect to this connection $\nabla_g$:

In [115]:
A_form = A.get_form(nab) # long time
A_form.display_expansion(E_U, stereoN)

A^(TU, nabla_g) = [1] + [0] + [0] + [0] + [1/24*(4*(a(t)^3 - a(t))*d(a)/dt - d(a)/dt*d^2(a)/dt^2)/pi^2 E^0/\E^1/\E^2/\E^3]

To attain $\hat{A}(TM, \nabla_g)$ in all given coordinates, we still have to extend the result onto $M$. With respect to the global frame $(\varepsilon_0, \varepsilon_1,\varepsilon_2,\varepsilon_3)$, the form $\hat{A}(TU, \nabla_g)$ only depends on the global coordinate $t$. This makes the continuation trivial. Besides, for the most part, one is interested in characteristic forms outside a set of measure zero. Hence, we terminate our calculation at this point.